In [1]:
import pandas as pd 
import re
import geonamescache
from unidecode import unidecode

In [2]:
f = open('./data/headlines.txt',"r")
headlines = f.read().splitlines()
f.close()

In [3]:
for headline in headlines:
    print(headline)

Zika Outbreak Hits Miami
Could Zika Reach New York City?
First Case of Zika in Miami Beach
Mystery Virus Spreads in Recife, Brazil
Dallas man comes down with case of Zika
Trinidad confirms first Zika case
Zika Concerns are Spreading in Houston
Geneve Scientists Battle to Find Cure
The CDC in Atlanta is Growing Worried
Zika Infested Monkeys in Sao Paulo
Brownsville teen contracts Zika virus
Mosquito control efforts in St. Louis take new tactics with Zika threat
San Juan reports 1st U.S. Zika-related death amid outbreak
Flu outbreak in Galveston, Texas
Zika alert – Manila now threatened
Zika afflicts 7 in Iloilo City
New Los Angeles Hairstyle goes Viral
Louisiana Zika cases up to 26
Orlando volunteers aid Zika research
Zika infects pregnant woman in Cebu
Chicago's First Zika Case Confirmed
Tampa Bay Area Zika Case Count Climbs
Bad Water Leads to Sickness in Flint, Michigan
Baltimore plans for Zika virus
London Health Unit Tracks Mad Cow Disease
Zika cases in Vietnam's Ho Chi Minh City su

In [4]:
gc = geonamescache.GeonamesCache()
citiesNorm = []
citiesOrg = []
city_unidecode = ""

for city in gc.get_cities().values():
    name = city['name']
    citiesOrg.append(city_unidecode)
    city_unidecode = unidecode(city['name']).replace("'", "")
    citiesNorm.append(city_unidecode)
    citiesOrg.append(city['name'])
        
citiesNorm.sort(key=lambda item: (-len(item), item), reverse=False)
citiesOrg.sort(key=lambda item: (-len(item), item), reverse=False)



_cities = "|".join(citiesNorm)
regex = "\\b(" + _cities + ")\\b"
print(regex)
compiled_re = re.compile(regex)

\b(Chak Two Hundred Forty-nine Thal Development Authority|Dolores Hidalgo Cuna de la Independencia Nacional|Ampliacion San Mateo (Colonia Solidaridad)|Licenciado Benito Juarez (Campo Gobierno)|Sant Pere, Santa Caterina i La Ribera|Nanchital de Lazaro Cardenas del Rio|Palikir - National Government Center|San Fernando del Valle de Catamarca|San Martin Texmelucan de Labastida|Acilia-Castel Fusano-Ostia Antica|Chak One Hundred Twenty Nine Left|el Camp den Grassot i Gracia Nova|Brandys nad Labem-Stara Boleslav|Delegacion Cuajimalpa de Morelos|Montecchio Maggiore-Alte Ceccato|Rosignano Solvay-Castiglioncello|Socorro Mission Number 1 Colonia|Sydney Central Business District|Jardines de la Silla (Jardines)|Marina di Ardea-Tor San Lorenzo|Parque Industrial Ciudad Mitras|Chak Five Hundred Seventy-five|Isernhagen Farster Bauerschaft|San Jose Guadalupe Otzacatipan|Santo Domingo de los Colorados|V.S.K.Valasai (Dindigul-Dist.)|Zurich (Kreis 2) / Wollishofen|Zurich (Kreis 6) / Unterstrass|Zurich (Kre

In [5]:
df = pd.DataFrame(columns =['headline', 'city', 'country']) 

In [6]:
for headline in headlines:
    m = compiled_re.search(headline)
    if m == None:
        df = df.append({'headline' : headline, "city" : "" , "country" : ""} , ignore_index=True)    
    else:  
        _city = m.group(0)
        citiesMatch = {}
        for city in citiesOrg: 
            if _city == unidecode(city).replace("'", ""):
                _cities = gc.get_cities_by_name(city)
                for _c in _cities:
                    for key, value in _c.items(): 
                        citiesMatch[key] = value
        biggestCity = {}
        for cityKey in citiesMatch:
            if len(biggestCity) == 0:
                biggestCity = citiesMatch[cityKey]
            else:
                if citiesMatch[cityKey]['population'] > biggestCity['population']:
                    biggestCity = citiesMatch[cityKey]
        df = df.append({'headline' : headline, "city" : biggestCity['name'] , "country" : biggestCity['countrycode']} , ignore_index=True)  
print(df)

                                              headline           city country
0                             Zika Outbreak Hits Miami          Miami      US
1                      Could Zika Reach New York City?  New York City      US
2                    First Case of Zika in Miami Beach    Miami Beach      US
3              Mystery Virus Spreads in Recife, Brazil         Recife      BR
4              Dallas man comes down with case of Zika         Dallas      US
..                                                 ...            ...     ...
645  Rumors about Rabies spreading in Jerusalem hav...      Jerusalem      IL
646              More Zika patients reported in Indang         Indang      PH
647  Suva authorities confirmed the spread of Rotav...           Suva      FJ
648         More Zika patients reported in Bella Vista    Bella Vista      DO
649                     Zika Outbreak in Wichita Falls  Wichita Falls      US

[650 rows x 3 columns]
